In [144]:
!pip install sklearn_crfsuite

In [145]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report
import nltk
import numpy as np
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [146]:

df = pd.read_csv('dataset.ptbr_harem.csv', delimiter=";", encoding = "UTF-8")
df.head(10)



,SENTENCE_ID,RELATION_ID,SENTENCE,ARGUMENT_1,ARGUMENT_1_CATEGORY,RELATION,ARGUMENT_2,ARGUMENT_2_CATEGORY
0,3,1,"Marcelo_Tejedor, paleontólogo de a Universidad...",Marcelo_Tejedor,PER,NaN,Universidade_Nacional_da_Patagônia,ORG
1,4,3,Esse é um alvo viável para remédios contra a o...,John_Clapham,PER,NaN,Smith_Kline_Beecham,ORG
2,9,2,Um grupo que reúne 13 sociedades científicas n...,Associação_Nacional_de_Biossegurança,ORG,NaN,ANBio,ORG
3,10,1,"Primeiro, pede que a CTNBio (Comissão_Técnica_...",CTNBio,ORG,NaN,Comissão_Técnica_Nacional_de_Biossegurança,ORG
4,11,1,A idéia foi lançada por o ministro_Roberto_Ama...,ministro_Roberto_Amaral,PER,NaN,Ciência_e_Tecnologia,ORG
5,11,5,A idéia foi lançada por o ministro_Roberto_Ama...,USP,ORG,NaN,Universidade_de_São_Paulo,ORG
6,12,1,"Francelino_Grando, secretário_de_Política_Tecn...",Francelino_Grando,PER,NaN,MCT,ORG
7,12,2,"Francelino_Grando, secretário_de_Política_Tecn...",secretário_de_Política_Tecnológica_Empresarial,PER,NaN,MCT,ORG
8,12,3,"Francelino_Grando, secretário_de_Política_Tecn...",MCT,ORG,NaN,Ministério_da_Ciência_e_Tecnologia,ORG
9,14,4,"Ainda ontem, o prefeito de Ribeirão, Gilberto_...",Gilberto_Maggioni,PER,NaN,PMN,ORG


In [0]:
class Entity():
   def __init__(self,name="NONE",categ='NONE'):
      self.name = name
      self.categ = categ
   def setName (self,name):
      self.name = name
   def getName(self):
      return self.name
   def setCategory(self,categ):
      self.categ = categ
   def getCategory(self):
      return self.categ
   def __str__(self):
      return "%s/%s"%(self.name,self.categ)
   def __eq__(self, other):
      if isinstance(other, Entity):
          return self.name == other.getName() and self.categ == other.getCategory()
      return False

class Relation():
   def __init__(self,arg1=None,rel=None,arg2=None):
      if type(arg1)==Entity and type(arg2)==Entity:
         self.arg1 = arg1
         self.rel = rel
         self.arg2 = arg2
      elif type(arg1)==str and type(arg2)==str:
         self.arg1 = Entity(name=arg1)
         self.rel = rel
         self.arg2 = Entity(name=arg2)
      elif (type(arg1)==tuple or type(arg1)==list)  and (type(arg2)==tuple or type(arg2)==list):
         self.arg1 = Entity(name=arg1[0],categ=arg1[1])
         self.rel = rel
         self.arg2 = Entity(name=arg2[0],categ=arg2[1])
   def setRel(self,rel):
     self.rel = rel
   def getArg1(self):
         return self.arg1
   def getArg2(self):
         return self.arg2
   def getRel(self):
         return self.rel
   def toTriple(self):
         return (self.arg1.getName(),self.rel,self.arg2.getName()) 
   def toQuintuple(self):
         return (self.arg1.getName(),self.arg1.getCategory(),self.rel,self.arg2.getName(),self.arg2.getCategory()) 
   def __str__(self):
      return "(%s,%s,%s)"%(str(self.arg1),self.rel,str(self.arg2))
   def __eq__(self, other):
      if isinstance(other, Relation):
          return self.arg1 == other.getArg1() and self.rel == other.getRel() and self.arg2 == other.getArg2()
      return False
   def __hash__(self):
      return hash(self.rel)

In [0]:
gold = [Relation((df["ARGUMENT_1"][0],df["ARGUMENT_1_CATEGORY"][0]),df["RELATION"][0],(df["ARGUMENT_2"][0],df["ARGUMENT_1_CATEGORY"][0]) )]

In [149]:
for go in gold:
  print((go.getArg1().getName(),go.getRel(),go.getArg2().getName()))

('Marcelo_Tejedor', nan, 'Universidade_Nacional_da_Patagônia')


In [0]:
import re
def normalize(text):
   text = text.lower()
   text = re.sub("de a","da",text)
   text = re.sub("de o","do",text)
   text = re.sub("de os ","dos",text)
   text = re.sub("de as ","das",text)
   text = re.sub("em a","na",text)
   text = re.sub("em o","no",text)
   text = re.sub("em os","nos",text)
   text = re.sub("em as", "nas",text)
   text = re.sub("á","a",text)
   text = re.sub("por a ","pela",text)
   text = re.sub("por o","pelo",text)
   text = re.sub("por os","pelos",text)
   text = re.sub("por as","pelas",text)
   text = re.sub("\((?! )","( ",text)
   text = re.sub("\r","",text)
   text = re.sub(",","",text)
   #text = re.sub("[^a-zA-z \n()0-9]","",text)
   text = re.sub("  ","",text)
   return text

In [0]:
text = df["SENTENCE"][0]
start = df["ARGUMENT_1"][0]
end = df["ARGUMENT_2"][0]
new_text = text[text.find(start)+len(start):text.rfind(end)]


In [152]:
print(new_text)

, paleontólogo de a 


In [0]:
new_text = normalize(new_text)


In [154]:
print(new_text)

 paleontólogo da 


In [0]:
go.setRel(new_text)

In [156]:
print(len(df))
gold = []

149


In [0]:
for i in range(len(df)):
 # print(df["ARGUMENT_1"][i])
  text = df["SENTENCE"][i]
  start = df["ARGUMENT_1"][i]
  end = df["ARGUMENT_2"][i]
  rel = text[text.find(start)+len(start):text.rfind(end)]
  rel = normalize(rel)
  gold.append(Relation((df["ARGUMENT_1"][i],df["ARGUMENT_1_CATEGORY"][i]),rel,(df["ARGUMENT_2"][i],df["ARGUMENT_1_CATEGORY"][i]) ))

In [158]:
for go in gold:
  print((go.getArg1().getName(),go.getRel(),go.getArg2().getName()))

('Marcelo_Tejedor', ' paleontólogo da ', 'Universidade_Nacional_da_Patagônia')
('John_Clapham', ' da empresa farmacêutica ', 'Smith_Kline_Beecham')
('Associação_Nacional_de_Biossegurança', ' ( ', 'ANBio')
('CTNBio', ' ( ', 'Comissão_Técnica_Nacional_de_Biossegurança')
('ministro_Roberto_Amaral', ' ( ', 'Ciência_e_Tecnologia')
('USP', ' ( ', 'Universidade_de_São_Paulo')
('Francelino_Grando', ' secretario_de_política_tecnológica_empresarial do ', 'MCT')
('secretário_de_Política_Tecnológica_Empresarial', ' do ', 'MCT')
('MCT', ' ( ', 'Ministério_da_Ciência_e_Tecnologia')
('Gilberto_Maggioni', ' ( ', 'PMN')
('Painel_Intergovernamental_de_Mudança_Climática', ' órgão da ', 'ONU')
('Gilson_Rambelli', ' paulo_bava_de_camargo e flavio_rizzi pesquisadores do ', 'Museu_de_Arqueologia_e_Etnologia')
('Flávio_Rizzi', ' pesquisadores do ', 'Museu_de_Arqueologia_e_Etnologia')
('Museu_de_Arqueologia_e_Etnologia', ' ( ', 'MAE')
('Museu_de_Arqueologia_e_Etnologia', ' ( mae) da ', 'USP')
('MAE', ') da ', 

In [0]:
#import joblib
#from nltk import word_tokenize

#teste_tagger = joblib.load('POS_tagger_brill.pkl')
#phrase = 'estão a ignorar as leis da ue em materia de seguranca maritima e nao se trata apenas da'
#tager = teste_tagger.tag(word_tokenize(phrase))

In [0]:
#print(tager)

In [161]:
!pip install spacy


In [162]:
!python -m spacy download pt
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
import spacy
nlp = spacy.load('pt')

In [0]:
doc = nlp(u'estão a ignorar as leis da')

In [0]:
my_list = [(token.orth_, token.pos_) for token in doc]

In [166]:
print(my_list)

[('estão', 'AUX'), ('a', 'ADP'), ('ignorar', 'VERB'), ('as', 'DET'), ('leis', 'NOUN'), ('da', 'ADJ')]


In [167]:
verb = "<ADV>*<AUX>*<VERB><PART>*<ADV>*|<NOUN|PROPN>*"
word = "<NOUN|ADJ|ADV|DET|ADP>"
preposition = "<ADP|ADJ>"

rel_pattern = "( %s (%s* (%s)+ )? )+ " % (verb, word, preposition)
grammar_long = '''REL_PHRASE: {%s}''' % rel_pattern

print (grammar_long)
reverb_pattern = nltk.RegexpParser(grammar_long)

REL_PHRASE: {( <ADV>*<AUX>*<VERB><PART>*<ADV>*|<NOUN|PROPN>* (<NOUN|ADJ|ADV|DET|ADP>* (<ADP|ADJ>)+ )? )+ }


In [0]:
rel = reverb_pattern.parse(my_list)

In [169]:
print(rel)

(S estão/AUX (REL_PHRASE a/ADP ignorar/VERB as/DET leis/NOUN da/ADJ))


In [170]:
rel_phrase = ""
for x in rel:
  if isinstance(x, nltk.Tree) and x.label() == 'REL_PHRASE':
    print(x)
    rel_phrase = " ".join([t[0] for t in x.leaves()])

(REL_PHRASE a/ADP ignorar/VERB as/DET leis/NOUN da/ADJ)


In [171]:
print(rel_phrase)

a ignorar as leis da


In [172]:
for go in gold:
  #print((go.getArg1().getName(),go.getRel(),go.getArg2().getName()))
  string = go.getRel()
  string.encode('utf-8')
  doc = nlp(string)
  #print(doc)
  my_list = [(token.orth_, token.pos_) for token in doc]
  rel = reverb_pattern.parse(my_list)
  rel_phrase = ""
  my_phrases  = []
  for x in rel:
    if isinstance(x, nltk.Tree) and x.label() == 'REL_PHRASE':
     # print(x)
      rel_phrase = ""
      rel_phrase = " ".join([t[0] for t in x.leaves()])
      my_phrases.append(rel_phrase)
  #print(my_phrases)
  if len(my_phrases)>0:
    relation = max(my_phrases,key=len)
  else:
    relation = " "
  print((go.getArg1().getName(),relation,go.getArg2().getName()))

('Marcelo_Tejedor', 'paleontólogo da', 'Universidade_Nacional_da_Patagônia')
('John_Clapham', 'da empresa farmacêutica', 'Smith_Kline_Beecham')
('Associação_Nacional_de_Biossegurança', ' ', 'ANBio')
('CTNBio', ' ', 'Comissão_Técnica_Nacional_de_Biossegurança')
('ministro_Roberto_Amaral', ' ', 'Ciência_e_Tecnologia')
('USP', ' ', 'Universidade_de_São_Paulo')
('Francelino_Grando', 'secretario_de_política_tecnológica_empresarial', 'MCT')
('secretário_de_Política_Tecnológica_Empresarial', ' ', 'MCT')
('MCT', ' ', 'Ministério_da_Ciência_e_Tecnologia')
('Gilberto_Maggioni', ' ', 'PMN')
('Painel_Intergovernamental_de_Mudança_Climática', 'órgão da', 'ONU')
('Gilson_Rambelli', 'flavio_rizzi pesquisadores', 'Museu_de_Arqueologia_e_Etnologia')
('Flávio_Rizzi', 'pesquisadores', 'Museu_de_Arqueologia_e_Etnologia')
('Museu_de_Arqueologia_e_Etnologia', ' ', 'MAE')
('Museu_de_Arqueologia_e_Etnologia', 'mae', 'USP')
('MAE', 'da', 'USP')
('Paulo_Bava_de_Camargo', 'flavio_rizzi pesquisadores', 'Museu_de_